In [7]:
import pandas as pd
import os
from sklearn.metrics import classification_report
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [8]:
import sys
sys.path.append('../src')

from models.classification_methods import get_classification_report

In [9]:
test_results_path = '../reports/test_results/'

list_df_t = os.listdir(test_results_path)
list_df_t.sort()
list_df_t

['DummyClassifier_bo_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_bo_users_Stance_test_results.csv',
 'DummyClassifier_bo_users_Timeline_test_results.csv',
 'DummyClassifier_cl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_cl_users_Stance_test_results.csv',
 'DummyClassifier_cl_users_Timeline_test_results.csv',
 'DummyClassifier_co_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_co_users_Stance_test_results.csv',
 'DummyClassifier_co_users_Timeline_test_results.csv',
 'DummyClassifier_gl_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_gl_users_Stance_test_results.csv',
 'DummyClassifier_gl_users_Timeline_test_results.csv',
 'DummyClassifier_ig_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_ig_users_Stance_test_results.csv',
 'DummyClassifier_ig_users_Timeline_test_results.csv',
 'DummyClassifier_lu_top_mentioned_timelines_Texts_test_results.csv',
 'DummyClassifier_lu_users_Stance_test_r

In [10]:
# Target list
target_list = [
    'ig',
    'bo', 
    'cl', 
    'co', 
    'gl', 
    'lu'
]

dict_cp = {
    'cl':'Hidroxicloroquina',
    'lu':'Lula',
    'co':'Sinovac',
    'ig':'Church',
    'gl':'Globo TV',
    'bo':'Bolsonaro',
}

names = list(dict_cp.values())
names

In [12]:
# (vectorizer,estimator, path_sring) 
results_tuples_stance = [
    ("-" ,"DummyClassifier", "DummyClassifier_{target}_users_Stance_test_results.csv"),
    ("TF-IDF" ,"xgboost", "XGBClassifier_TfidfVectorizer_cl_users_Stance_test_results.csv"),
    ("bertabaporu-base-uncased" ,"xgboost", "XGBClassifier_TfidfVectorizer_cl_users_Stance_test_results.csv"),
    ("-" ,"bertabaporu-base-uncased", "bert_classifier_pablocosta_bertabaporu_base_uncased_{target}_Stance_test_results.csv")
    
    
]

list_results = []
for vectorizer, estimator, path_results in results_tuples_stance:
    
    list_cr = []
    
    for target in target_list:
        
        df_results = pd.read_csv(test_results_path + path_results.format(target = target))
        
        
        # get classification report df
        df_classification_report = get_classification_report(df_results.test, df_results.pred, cr_args = {})
        
        # create multindex
        column_indexes = [(metric,dict_cp[target]) for metric in df_classification_report.columns]
        multi_index_cols = pd.MultiIndex.from_tuples(column_indexes, names=['metric', 'target'])
        rows_indexes = [(vectorizer, estimator, cl) for cl in df_classification_report.index]
        multi_index_rows = pd.MultiIndex.from_tuples(rows_indexes, names=['vectorizer', 'estimator', 'class'])
        df_classification_report.columns = multi_index_cols
        df_classification_report.index = multi_index_rows
        
        
        
        list_cr.append(df_classification_report)
        
    df_results = pd.concat(list_cr, axis = 1)
    
    list_results.append(df_results)
    
df_results_final = pd.concat(list_results)

In [13]:
df_results_final

metric                                                         precision  \
target                                                            Church   
vectorizer               estimator                class                    
-                        DummyClassifier          against       0.565943   
                                                  accuracy      0.565943   
                                                  weighted avg  0.320292   
                                                  macro avg     0.282972   
                                                  for           0.000000   
TF-IDF                   xgboost                  against       0.716088   
                                                  accuracy      0.735192   
                                                  weighted avg  0.737273   
                                                  macro avg     0.737422   
                                                  for           0.758755   
bertabaporu-base-uncased xgboost                  against       0.716088   
                                                  accuracy      0.735192   
                                                  weighted avg  0.737273   
                                                  macro avg     0.737422   
                                                  for           0.758755   
-                        bertabaporu-base-uncased against       0.850267   
                                                  accuracy      0.871452   
                                                  weighted avg  0.874748   
                                                  macro avg     0.878467   
                                                  for           0.906667   

metric                                                            recall  \
target                                                            Church   
vectorizer               estimator                class                    
-                        DummyClassifier          against       1.000000   
                                                  accuracy      0.565943   
                                                  weighted avg  0.565943   
                                                  macro avg     0.500000   
                                                  for           0.000000   
TF-IDF                   xgboost                  against       0.785467   
                                                  accuracy      0.735192   
                                                  weighted avg  0.735192   
                                                  macro avg     0.734839   
                                                  for           0.684211   
bertabaporu-base-uncased xgboost                  against       0.785467   
                                                  accuracy      0.735192   
                                                  weighted avg  0.735192   
                                                  macro avg     0.734839   
                                                  for           0.684211   
-                        bertabaporu-base-uncased against       0.938053   
                                                  accuracy      0.871452   
                                                  weighted avg  0.871452   
                                                  macro avg     0.861334   
                                                  for           0.784615   

metric                                                          f1-score  \
target                                                            Church   
vectorizer               estimator                class                    
-                        DummyClassifier          against       0.722814   
                                                  accuracy      0.565943   
                                                  weighted avg  0.409072   
                                              

In [14]:
mask_f1 = [True if  "f1-score" in col else False for col in df_results_final.columns]
mask_macro = [True if  "macro avg" in col else False for col in df_results_final.index]

In [26]:
f1_macro_df = df_results_final.loc[mask_macro,mask_f1]
f1_macro_df[('f1-score','overall')] = f1_macro_df.mean(axis=1)

f1_macro_df

metric                                                       f1-score  \
target                                                         Church   
vectorizer               estimator                class                 
-                        DummyClassifier          macro avg  0.361407   
TF-IDF                   xgboost                  macro avg  0.734366   
bertabaporu-base-uncased xgboost                  macro avg  0.734366   
-                        bertabaporu-base-uncased macro avg  0.866621   

metric                                                                 \
target                                                      Bolsonaro   
vectorizer               estimator                class                 
-                        DummyClassifier          macro avg  0.462857   
TF-IDF                   xgboost                  macro avg  0.734366   
bertabaporu-base-uncased xgboost                  macro avg  0.734366   
-                        bertabaporu-base-uncased macro avg  0.739125   

metric                                                                         \
target                                                      Hidroxicloroquina   
vectorizer               estimator                class                         
-                        DummyClassifier          macro avg          0.334878   
TF-IDF                   xgboost                  macro avg          0.734366   
bertabaporu-base-uncased xgboost                  macro avg          0.734366   
-                        bertabaporu-base-uncased macro avg          0.844778   

metric                                                                 \
target                                                        Sinovac   
vectorizer               estimator                class                 
-                        DummyClassifier          macro avg  0.351759   
TF-IDF                   xgboost                  macro avg  0.734366   
bertabaporu-base-uncased xgboost                  macro avg  0.734366   
-                        bertabaporu-base-uncased macro avg  0.840499   

metric                                                                 \
target                                                       Globo TV   
vectorizer               estimator                class                 
-                        DummyClassifier          macro avg  0.372519   
TF-IDF                   xgboost                  macro avg  0.734366   
bertabaporu-base-uncased xgboost                  macro avg  0.734366   
-                        bertabaporu-base-uncased macro avg  0.862638   

metric                                                                 \
target                                                           Lula   
vectorizer               estimator                class                 
-                        DummyClassifier          macro avg  0.344578   
TF-IDF                   xgboost                  macro avg  0.734366   
bertabaporu-base-uncased xgboost                  macro avg  0.734366   
-                        bertabaporu-base-uncased macro avg  0.796338   

metric                                                                 
target                                                        overall  
vectorizer               estimator                class                
-                        DummyClassifier          macro avg  0.371333  
TF-IDF                   xgboost                  macro avg  0.734366  
bertabaporu-base-uncased xgboost                  macro avg  0.734366  
-                        bertabaporu-base-uncased macro avg  0.825000